In [1]:
struct Course
    name::String
end

mutable struct Relation
    rtype::String
    members::Array{Union{Course, Relation}, 1}
    Relation(rtype::String, members::Array) =  new(uppercase(strip(rtype, [' ', '('])), members)
end 

function Relation(inpt)
    if isa(inpt, String)
        return Relation(uppercase(strip(inpt, [' ', '('])), [])
    elseif isa(inpt, Array) && eltype(inpt) in (Course, Relation)
        return Relation("", inpt)
    else
        return error("either rtype or members have incorrect type")
    end
end

Relation() = Relation("", [])

Relation

In [ ]:
function pprint(r)
    for elem in r.members
        

In [5]:
d = ["Math 1315", "or (", "Math 1410", "and", "Math 1415", "or (", "Math 123", "and", "Math456", ")", ")",
    "or", "Math 1530", "Reading & w p 2"]

14-element Vector{String}:
 "Math 1315"
 "or ("
 "Math 1410"
 "and"
 "Math 1415"
 "or ("
 "Math 123"
 "and"
 "Math456"
 ")"
 ")"
 "or"
 "Math 1530"
 "Reading & w p 2"

In [ ]:
[or(or(m1315, or(and(m1410, m1415), and(m123, m456))), m15130), rwp2
    
]

In [2]:
d = ["Math 1315", "or (", "Math 1410", "and", "Math 1415", "or (", "Math 123", "and", "Math456", ")", ")",
    "or", "Math 1530", "Reading & w p 2"]

14-element Vector{String}:
 "Math 1315"
 "or ("
 "Math 1410"
 "and"
 "Math 1415"
 "or ("
 "Math 123"
 "and"
 "Math456"
 ")"
 ")"
 "or"
 "Math 1530"
 "Reading & w p 2"

In [7]:
function itre(it::Relation)
    for i in it.members
        if isa(i, Relation)
           itre(i)
        elseif isa(i, Course)
            print(i.name, ',')
            if it.rtype == "AND"
                print(hash(it), ',', '2', '\n')
            else
                print(hash(i.name), ',', '1','\n')
            end
        end
    end
end

for r in reqs
    if isa(r, Course)
        print(r.name, ',', hash(r.name), ',', '0', '\n')
    else
        itre(r)
    end
end

Math 1315,2263969766702654586,1
Math 1410,3013351850941791963,2
Math 1415,3013351850941791963,2
Math 123,4919044556118090714,2
Math456,4919044556118090714,2
Math 1530,955886123823477966,1
Reading & w p 2,7703009946950956736,0


why


function itre(it)
    for i in it[].members
        if isa(i, Base.RefValue{Relation})
           itre(i)
        elseif isa(i, Base.RefValue{Course})
            print(i[].name, ',')
            if it[].rtype == "AND"
                print(hash(it[]), ',', '2', '\n')
            else
                print(hash(i[].name), ',', '1','\n')
            end
        end
    end
end

for r in reqs
    if isa(r, Base.RefValue{Course})
        print(r[].name, ',', hash(r[].name), ',', '0', '\n')
    else
        itre(r)
    end
end

function pushrel!(reqs::Array, input, elem)
    # push a standard relation to reqs
    if isa(reqs[end], Base.RefValue{Relation}) && reqs[end][].rtype == uppercase(strip(elem, [' ', '('])) # if relation and rtype same as current elem
        topush = Course(popfirst!(input))
        push!(reqs[end][].members, Ref(topush))
    else
        topush = Relation(elem, [reqs[end], Ref(Course(popfirst!(input)))])
        pop!(reqs)
        push!(reqs, Ref(topush))
    end
end

function pushrel!(reqs::Base.RefValue{Relation}, input, elem)
    # push a standard relation to reqs
    if isa(reqs[].members[end], Base.RefValue{Relation}) && reqs[].members[end][].rtype  == uppercase(strip(elem, [' ', '('])) # if relation and rtype same as current elem
        topush = Course(popfirst!(input))
        push!(reqs[end][].members, Ref(topush))
        
    else
        topush = Relation(elem, [reqs[].members[end], Ref(Course(popfirst!(input)))])
        pop!(reqs[].members)
        push!(reqs[].members, Ref(topush))
    end
end

function pushgroup!(reqs::Array, input, elem)
    rel = Relation(elem, [reqs[end]]) 
    pop!(reqs)
    while input[1] != ")"
        elem = popfirst!(input)
       if lowercase(elem) in ("or", "and")
            pushrel!(Ref(rel), input, elem)
        elseif lowercase(elem) in ("or (", "and (")
            pushgroup!(Ref(rel), input, elem)
        else
            push!(rel.members, Ref(Course(elem)))
        end
    end
    push!(reqs, Ref(rel))
    popfirst!(input)
end

function pushgroup!(reqs::Base.RefValue{Relation}, input, elem)
    rel = Relation(elem, [reqs[].members[end]]) 
    pop!(reqs[].members)
    while input[1] != ")"
        elem = popfirst!(input)
       if lowercase(elem) in ("or", "and")
            pushrel!(Ref(rel), input, elem)
        elseif lowercase(elem) in ("or (", "and (")
            pushgroup!(rel, input, elem)
        else
            push!(rel.members, Ref(Course(elem)))
        end
    end
    push!(reqs[].members, Ref(rel))
    popfirst!(input)
end

need relation int 

In [3]:
function pushrel!(reqs::Array, input, elem)
    # push a standard relation to reqs
    if isa(reqs[end], Relation) && reqs[end].rtype == uppercase(strip(elem, [' ', '('])) # if relation and rtype same as current elem
        topush = Course(popfirst!(input))
        push!(reqs[end].members, topush)
    else
        topush = Relation(elem, [reqs[end], Course(popfirst!(input))])
        pop!(reqs)
        push!(reqs, topush)
    end
end

function pushrel!(reqs::Relation, input, elem)
    # push a standard relation to reqs
    if isa(reqs.members[end], Relation) && reqs.members[end].rtype  == uppercase(strip(elem, [' ', '('])) # if relation and rtype same as current elem
        topush = Course(popfirst!(input))
        push!(reqs[end].members, topush)
        
    else
        topush = Relation(elem, [reqs.members[end], Course(popfirst!(input))])
        pop!(reqs.members)
        push!(reqs.members, topush)
    end
end

function pushgroup!(reqs::Array, input, elem)
    rel = Relation(elem, [reqs[end]]) 
    pop!(reqs)
    while input[1] != ")"
        elem = popfirst!(input)
       if lowercase(elem) in ("or", "and")
            pushrel!(rel, input, elem)
        elseif lowercase(elem) in ("or (", "and (")
            pushgroup!(rel, input, elem)
        else
            push!(rel.members, Course(elem))
        end
    end
    push!(reqs, rel)
    popfirst!(input)
end

function pushgroup!(reqs::Relation, input, elem)
    rel = Relation(elem, [reqs.members[end]]) 
    pop!(reqs.members)
    while input[1] != ")"
        elem = popfirst!(input)
       if lowercase(elem) in ("or", "and")
            pushrel!(rel, input, elem)
        elseif lowercase(elem) in ("or (", "and (")
            pushgroup!(rel, input, elem)
        else
            push!(rel.members, Course(elem))
        end
    end
    push!(reqs.members, rel)
    popfirst!(input)
end

pushgroup! (generic function with 2 methods)

In [6]:
input = copy(d)
reqs = []
while !isempty(input)
    c = popfirst!(input)
    if lowercase(c) in ("or", "and")
        pushrel!(reqs, input, c)
        continue
    elseif lowercase(c) in ("or (", "and (")
        pushgroup!(reqs, input, c)
        continue
    else
        push!(reqs, Course(c))
   end
end

        

In [3]:
using CSV, DataFrames

In [4]:
dat = CSV.read("../py/programs_and_requirements.txt", DataFrame, delim='|')
dat = coalesce.(dat, "");

In [3]:
replaceescchar(str) = lowercase(replace(unescape_string(str), "\xa0"=>" ", "'"=>"", "\t"=>"", ","=>" |"))

replacesubstr(str) = replace(str, "+"=>"and", "or equivalent"=>"", "or equivalent course"=>"")

r = Regex("(?<andor>(or|and))\\s\\s\\|\\s\\(")
s = SubstitutionString("\\g<andor> (")
r2 = Regex("(?<paror>\\)\\s)(?<oa>(or|and))")
s2 = SubstitutionString("\\g<paror> | \\g<oa>")
fixgroup(str) = replace(str,r=>s, r2=>s2)

function regexextract(str)
    # crssubjnumb = "(\\w\\w\\w?\\w?\\s\\d\\d\\d\\d?\\w?\\s(?=\\|))"
    crssubjnumb = "(\\w{2,4}\\s\\d{3,4}\\w?\\s(?=\\|))"
    andor = "(\\sor\\s\\(.*\\))|(\\sand\\s\\(.*\\))|(\\sor\\s)|(\\sand\\s)"
    parenth = "[()]"
    #skills = "(.*\\sskills\\s\\d)"
    skills = "(?<=\\|\\s)[\\w\\s&]{4,}\\sskills\\s\\d?"
    series = "(?<=\\|\\s)[\\w\\s\\-\\d]{4,}\\sseries"
    proficiency = "(?<=\\|\\s)[\\w\\s\\-&\\d]{4,}\\d?\\sproficiency\\s\\d?"
    divider = "(\\|(=? \\||))"
    return join(map(x->x.match, eachmatch(Regex("$crssubjnumb|$andor|$parenth|$skills|$series|$proficiency|$divider"), str)))
end

function chopper(str)
    if startswith(str, "|") && endswith(str, "|")
        return chop(str, head = 1, tail = 1)
    elseif startswith(str, "|")
        return chop(str, head = 1, tail = 0)
    elseif startswith(str, "or")
        return chop(str, head = 2, tail = 0)
    elseif startswith(str, "and")
        return chop(str, head = 3, tail = 0)
    elseif endswith(str, "|")
        return chop(str, head = 0, tail = 1)
    elseif endswith(str, "or")
        return chop(str, head = 0, tail = 2)
    elseif endswith(str, "and")
        return chop(str, head = 0, tail = 3)
    else
        return ""
    end
end

function catorand(x, y)
    if (occursin("or ", x) || occursin("and ", x)) && occursin("(", y)
        return x*y
    end
end

finfiltr(arr) = filter(x->!isempty(strip(x))&&!occursin("()",x), arr)

function processor(str)
    rec = replaceescchar(str)
    rss = replacesubstr(rec)
    fg = fixgroup(rss)
    re = strip(regexextract(fg))
    c = re
    while true
        toret = c
        c = chopper(strip(c))
        if isempty(c) 
            return finfiltr(split(toret,"|", keepempty=false))
        end
    end
end

processor (generic function with 1 method)

In [37]:
match(r"\w{2,4}\s\d{3,4}\w?", dd)

In [28]:
function p(str)
    rec = replaceescchar(str)
    fg = fixgroup(rec)
    re = strip(regexextrct(fg))
    return re
end
dd = p(fixdata[:, 2][1])

function regexextract(str)
    # TODO crssubjnumb misses hlth 1001 for comm dental hlth coord cert 1618
    # TODO need to fix chopper, or move stripping strngs ending in `or` somewhere else.
    # if a string ends in `or` it is not valid but `or` is kept.
    # examples: cdhc and computer information systems cyber sec 1618. maybe just add another
    # filter function to get rid of or/and at end?
    crssubjnumb = "(\\w{2,4}\\s\\d{3,4}\\w?\\s(?=\\|))"
    andor = "(\\sor\\s\\(.*\\))|(\\sand\\s\\(.*\\))|(\\sor\\s)|(\\sand\\s)"
    parenth = "[()]"
    skills = "(?<=\\|\\s)[\\w\\s&]{4,}\\sskills\\s\\d?"
    series = "(?<=\\|\\s)[\\w\\s\\-\\d]{4,}\\sseries"
    proficiency = "(?<=\\|\\s)[\\w\\s\\-&\\d]{4,}\\d?\\sproficiency\\s\\d?"
    divider = "(\\|(=? \\||))"
    return join(map(x -> x.match, eachmatch(Regex("$crssubjnumb|$andor|$parenth|$skills|$series|$proficiency|$divider"), str)))
end

regexextract(dd)

"||||||| or ||() and |"

In [58]:
d = fixdata[:, 2][1]
println(d)
output = regexextract(fixgroup(replaceescchar(d)))
c = output
println(c)
while true
        toret = c
        c = chopper(strip(c))
        println(c)
        if isempty(c)
            arr = finfiltr(split(toret, "|", keepempty=false))
            println(convert(Array{String},collect(map(strip, arr))))
            break
    end
end

['Associate degree or higher in a technical field, or', 'Department approval']
 or | or |
 | or |
 or 

["or"]


In [54]:
replaceescchar(str) = lowercase(replace(unescape_string(str),
    "\xa0" => " ",
    "'" => "",
    "\t" => "",
    "," => " |",
    "+" => "and")
)

finfiltr(arr) = filter(x -> !isempty(strip(x)) && !occursin("()", x), arr)

function fixgroup(str)
    r = Regex("(?<andor>(or|and))\\s\\s\\|\\s\\(")  # replace `or | (` with `or (`
    s = SubstitutionString("\\g<andor> (")          # converts to single element  in final arr
    r2 = Regex("(?<paror>\\)\\s)(?<oa>(or|and))")   # replace `) or` with `) | or`
    s2 = SubstitutionString("\\g<paror> | \\g<oa>") # converts to two seperate elements in final array
    return replace(str, r => s, r2 => s2)
end

function regexextract(str)
    # TODO crssubjnumb misses hlth 1001 for comm dental hlth coord cert 1618
    # TODO need to fix chopper, or move stripping strngs ending in `or` somewhere else.
    # if a string ends in `or` it is not valid but `or` is kept.
    # examples: cdhc and computer information systems cyber sec 1618. maybe just add another
    # filter function to get rid of or/and at end?
    crssubjnumb = "(\\w{2,4}\\s\\d{3,4}\\w?\\s(?=(\\||\\-)))"
    andor = "(\\sor\\s\\(.*\\))|(\\sand\\s\\(.*\\))|(\\sor\\s)|(\\sand\\s)"
    parenth = "[()]"
    skills = "(?<=\\|\\s)[\\w\\s&]{4,}\\sskills\\s\\d?"
    series = "(?<=\\|\\s)[\\w\\s\\-\\d]{4,}\\sseries"
    proficiency = "(?<=\\|\\s)[\\w\\s\\-&\\d]{4,}\\d?\\sproficiency\\s\\d?"
    divider = "(\\|(=? \\||))"
    return join(map(x -> x.match, eachmatch(Regex("$crssubjnumb|$andor|$parenth|$skills|$series|$proficiency|$divider"), str)))
end

function chopper(str)
    if startswith(str, "|") && endswith(str, "|")
        return chop(str, head=1, tail=1)
    elseif startswith(str, "|")
        return chop(str, head=1, tail=0)
    elseif startswith(str, "or")
        return chop(str, head=2, tail=0)
    elseif startswith(str, "and")
        return chop(str, head=3, tail=0)
    elseif endswith(str, "|")
        return chop(str, head=0, tail=1)
    elseif endswith(str, "()")
        return chop(str, head=0, tail=2)
    elseif endswith(str, "or")
        return chop(str, head=0, tail=2)
    elseif endswith(str, "and")
        return chop(str, head=0, tail=3)
    else
        return ""
    end
end

function runclean(str)
    rec = replaceescchar(str)
    fg = fixgroup(rec)
    re = strip(regexextrct(fg))
    c = re
    while true
        toret = c
        c = chopper(strip(c))
        if isempty(c)
            arr = finfiltr(split(toret, "|", keepempty=false))
            return convert(Array{String},collect(map(strip, arr)))
        end
    end
end


runclean (generic function with 1 method)

In [6]:
f(x) = convert(Array{String}, runclean(x))
transform!(dat, :requirements=>ByRow(f)=>"cleanedreqs");

In [56]:
fixdata = filter(row->occursin("Computer Information Systems (Post Degree Certificate of Completion)  Cyber Security Concentration", row.program)&&row.year==1618, dat)

,program
,String
1,Program: Computer Information Systems (Post Degree Certificate of Completion) Cyber Security Concentration


In [ ]:
using Random

In [5]:
rws = [rand(1:nrow(dat)) for _ in 1:10]

10-element Vector{Int64}:
 248
 869
 949
 475
  21
 947
 256
 738
 401
 378

In [6]:
dat[rws,[:requirements]]

,requirements
,String
1,"['\\n\\t', 'High School Diploma or Equivalent', '\\n\\t', 'Math Proficiency 2', '\\n\\t', 'Reading & Writing Proficiency 2', '\\n\\t', 'Biology Proficiency', '\\n']"
2,"['\\n\\t', 'Math Skills 2', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
3,"['\\n\\t', 'Math Skills 2', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
4,"['\\n\\t', 'Math Skills 2', '\\n\\t', 'Reading & Writing Skills 2', 'or', 'ENG 1101', 'or', 'ENG 1101P', '(Recommended) *', '\\n\\t', 'CHEM 1410', 'and', 'CHEM 1492', '\\n']"
5,"['\\n\\t', 'Math 1111-1114 Series', 'or', 'Math Skills 3', 'or', 'Math Skills 4', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
6,"['\\n\\t', 'Math Skills 5', 'or', 'Math Skills 6', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
7,"['\\n\\t', 'MATH 1210', 'or ', 'MATH 1310', 'or ', 'Math Proficiency 3', '\\n\\t', 'Reading & Writing Proficiency 2', '\\n']"
8,"['\\n\\t', 'High School Diploma or Equivalent for employment', '\\n\\t', 'At least 18 years of age to attend IFSAC Certification Courses', '\\n\\t', '\\n\\t', 'Math Skills 2', '\\n\\t', '\\n\\t', '\\n\\t', 'Reading & Writing Skills 2', '\\n\\t\\xa0\\xa0', '\\n']"
9,"['\\n\\t', 'MATH 1310', '\\n\\t', 'Reading & Writing Proficiency 2', '\\n']"


In [222]:
dat[rws,[:cleanedreqs]]

,cleanedreqs
,Array…
1,"[""math skills 4"", ""reading & writing skills 2""]"
2,[]
3,"[""math 1111-1114 series"", "" or "", ""math skills 3"", "" or "", ""math skills 4"", ""reading & writing skills 2""]"
4,"[""math skills 2"", ""reading & writing skills 2""]"
5,"[""math proficiency 3"", ""reading & writing proficiency 2""]"
6,"[""math skills 2"", ""reading & writing skills 2""]"
7,[]
8,"[""reading & writing skills 1""]"
9,[]


In [38]:
dd

"|math 2 proficiency ||reading & writing proficiency 2|||| or ||() and |"

In [278]:
filter(row->ismissing(row.requirements),dat)[1, 2]

missing

In [376]:
#f(dat.requirements[end])
dat[11,2]

"['\\n\\t', 'Math Skills 4', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"

In [370]:
filter(row->occursin("Community Dental", row.program), dat)[:, :cleanedreqs]

3-element Vector{Vector{String}}:
 [" math 2 proficiency", " reading & writing proficiency", "hlth 1001 ", " or "]
 [" math skills 2", " reading & writing skills 2"]
 [" math skills 2", " reading & writing skills 2"]

In [ ]:
transform(dat, :cleanreqs=>ByRow(x-> ))

In [358]:
 filter(row->occursin("(", join(row.cleanedreqs)), dat)[28, :cleanedreqs])

4-element Vector{String}:
 " math skills 2 "
 " or "
 " math skills 3"
 " reading & writing skills 2"

In [255]:
ENV["COLUMNS"] = 120

120

In [265]:
filter(row -> occursin("and", join(row.cleanedreqs)), dat)[1, :]

6-element Vector{String}:
 "tlol 1010  and "
 "tlol 1015  and "
 "tlol 1020 "
 "tlol 1025 "
 "tlol 1030  and "
 "tlol 1035 "

In [213]:
occursin("(", join(["a", "("]))

true

In [209]:
last(dat.requirements, 10)

10-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "['\\n\\t', 'Alternative Teacher L" ⋯ 41 bytes ⋯ "\\t', 'Bachelor’s Degree', '\\n']"
 "['\\n\\t', 'Alternative Teacher L" ⋯ 41 bytes ⋯ "\\t', 'Bachelor’s Degree', '\\n']"
 "['\\n\\t', 'Alternative Teacher L" ⋯ 41 bytes ⋯ "\\t', 'Bachelor’s Degree', '\\n']"
 "['GECK 0500 - Computer Keyboard" ⋯ 641 bytes ⋯ "dies ', '0', 'credit hour(s)']"
 "['\\n\\t', 'Math Skills 2', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
 "['Bachelor’s Degree', '\\n', 'De" ⋯ 127 bytes ⋯ " 'or equivalent course', '\\n']"
 "['\\n\\t', 'Math Skills 3', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
 "['\\n\\t', 'Math Skills 2', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
 "['\\n\\t', 'Math Skills 3', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"
 "['\\n\\t', 'Math Skills 3', '\\n\\t', 'Reading & Writing Skills 2', '\\n']"

In [100]:
crssubjnumb = "(\\w\\w\\w?\\w?\\s\\d\\d\\d\\d?\\s)"
    andor = "(\\sor\\s)|(\\sand\\s)"
    skills = "(.*\\sskills\\s\\d)"
    proficiency = "(.*\\sproficiency)"
    divider = "(\\|)"
[eachmatch(Regex("$crssubjnumb|$andor|$skills|$proficiency|$divider"), dat.cleanedreqs[9])...]

29-element Vector{RegexMatch}:
 RegexMatch("at 1005 ", 1="at 1005 ", 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6=nothing)
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch(" or ", 1=nothing, 2=" or ", 3=nothing, 4=nothing, 5=nothing, 6=nothing)
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch("at 1010 ", 1="at 1010 ", 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6=nothing)
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing, 5=nothing, 6="|")
 RegexMatch(" or ", 1=nothing, 2=" or ", 3=nothing, 4=nothing, 5=nothing, 6=nothing)
 RegexMatch("|", 1=nothing, 2=nothing, 3=nothing, 4=nothing,

In [83]:
 join(map(x->x.match, eachmatch(r"(\w\w\w?\w?\s\d\d\d\d?\s)|(\sor\s)|(\sand\s)", replaceescchar(dat.requirements[9]))))


"at 1005  or at 1010  or at 1020  or at 1030  or at 1040 "

In [65]:
dat.requirements[9][352:end]

"ortation ', '3', 'credit hour(s)', 'Program Approved Electives', ' 12 credit hour(s)']"

In [20]:
for x in dat.cleanedreqs[9]
    if occursin('-', x)
        m = match(r".*((?=\W-))", x)
        println(m.match)
    elseif isdigit(strip(x)) || occursin("credit hours", x)
        pop
    end
end

at 1005
 at 1010
 at 1020
 at 1030
 at 1040


In [22]:
const validset = Set(['1', '2','3','4','5'])
filter(:cleanedreqs => in(validset), dat)

,program,requirements,year,cleanedreqs
,String,String,Int64,Array…
